# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

In [1]:
!pip install --q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.7/160.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 5.

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [3]:
from crewai import Agent, Task, Crew

In [ ]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
llm=ChatGroq(temperature=0,model_name="llama3-70b-8192",api_key=api_key)

The workflow is going to be like this 

It will consist of 3 agents
1. Planner
2. Writer
3. Editor

 So each agent is going to assigned a specific role , goal , backstory

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
    llm=llm,
	verbose=True
)

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    llm=llm,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

Now create tasks for each agent explaining what is the description - what it needs to do , what is teh expected output and name of the agent


### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
from IPython.display import Markdown

In [15]:
topic = "Moon landing"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Moon landing.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer!

Final Answer:

**Content Plan: "Uncovering the Wonders of the Moon Landing"**

**Target Audience:**

* Demographics: Space enthusiasts, history buffs, students, and individuals interested in science and technology
* Interests: Space exploration, NASA, Apollo missions, moon landing, space travel, astronomy
* Pain points: Limited knowledge about the moon landing, seeking accurate and engaging information, wanting to stay updated on the latest developments in space exploration

**Content Outline:**

I. Introduction

* Hook: "One small s

In [16]:
Markdown(result)

**Uncovering the Wonders of the Moon Landing**
=====================================================

### I. Introduction

"One small step for man, one giant leap for mankind" - these iconic words, spoken by Neil Armstrong, still resonate with us today. The moon landing, a historic event that marked a major milestone in human space exploration, continues to fascinate and inspire us. As we reflect on this remarkable achievement, we're reminded of the power of human ingenuity, collaboration, and determination. In this article, we'll delve into the history, significance, and impact of the moon landing, exploring its lasting legacy on modern space exploration.

The moon landing, which took place on July 20, 1969, was a groundbreaking moment that captivated the world's attention. As we celebrate this remarkable achievement, we're also reminded of the significance of space exploration and its potential to shape our future. In this article, we'll examine the key players, events, and discoveries that made the moon landing possible, as well as its ongoing impact on science, technology, and human progress.

### II. The History of the Moon Landing

The moon landing was a culmination of years of hard work, dedication, and innovation by NASA, the Apollo 11 crew, and thousands of individuals involved in the Apollo program. The journey began on July 16, 1969, when the Apollo 11 spacecraft, carrying Neil Armstrong, Buzz Aldrin, and Michael Collins, launched from Kennedy Space Center in Florida. After traveling through space for four days, the spacecraft entered into lunar orbit, preparing for the historic landing.

On July 20, Armstrong and Aldrin suited up in their spacesuits, climbed into the lunar module Eagle, and descended onto the moon's surface. At 2:56 UTC, Armstrong radioed back to Mission Control on Earth, "Houston, Tranquility Base here. The Eagle has landed." Six hours later, Armstrong made history by becoming the first person to set foot on the moon, famously declaring, "That's one small step for man, one giant leap for mankind." This remarkable achievement marked a major breakthrough in space exploration, demonstrating the power of human ingenuity and collaboration.

### III. The Significance of the Moon Landing

The moon landing was a monumental achievement that marked a major breakthrough in space exploration. The mission demonstrated the power of human ingenuity, showcasing our ability to overcome seemingly insurmountable challenges. The scientific discoveries and technological advancements that emerged from the Apollo program have had a lasting impact on our understanding of space and the universe. The moon landing also inspired a generation of scientists, engineers, and explorers, sparking a renewed interest in space exploration and STEM education.

The mission's success was a testament to international cooperation, as scientists and engineers from around the world collaborated to achieve this remarkable feat. The moon landing's significance extends beyond the Apollo program, shaping our understanding of space, driving innovation, and inspiring future generations of scientists, engineers, and explorers.

### IV. The Legacy of the Moon Landing

Today, the moon landing continues to shape our understanding of space and our place in the universe. NASA's ongoing research and exploration efforts are building upon the foundation laid by the Apollo program. Private companies like SpaceX and Blue Origin are pushing the boundaries of space travel and exploration, driven by the same spirit of innovation and discovery that defined the moon landing. The moon landing's legacy serves as a powerful reminder of human potential and achievement, inspiring us to continue exploring, to push beyond the boundaries of what's thought possible, and to strive for greatness.

### V. Conclusion

The moon landing was a historic event that continues to captivate and inspire us today. As we reflect on this remarkable achievement, we're reminded of the power of human ingenuity, collaboration, and determination. The moon landing's legacy extends far beyond the Apollo program, shaping our understanding of space, driving innovation, and inspiring future generations of scientists, engineers, and explorers. As we look to the future, we're encouraged to stay curious, to ask questions, and to seek answers. The moon landing serves as a powerful reminder of what we can achieve when we work together, pushing the boundaries of what's thought possible. Let's continue to explore, to discover, and to reach for the stars.

**SEO Keywords:** Moon landing, Apollo 11, NASA, Neil Armstrong, Buzz Aldrin, Space exploration, Space travel, Astronomy, Space history, Historic events.